### 작업1 유형
데이터 출처 : https://archive.ics.uci.edu/ml/datasets/Bank+Marketing (후처리 작업)  
데이터 설명 : 은행의 전화 마케팅에 대해 고객의 반응 여부  
dataurl : https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/train.csv

In [54]:
import numpy as np
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/train.csv')
print(df.shape)
df.head()

(12870, 17)


,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome,y
0,13829,29,technician,single,tertiary,no,18254,no,no,cellular,11,may,2,-1,0,unknown,no
1,22677,26,services,single,secondary,no,512,yes,yes,unknown,5,jun,3,-1,0,unknown,no
2,10541,30,management,single,secondary,no,135,no,no,cellular,14,aug,2,-1,0,unknown,no
3,13689,41,technician,married,unknown,no,30,yes,no,cellular,10,jul,1,-1,0,unknown,no
4,11304,27,admin.,single,secondary,no,321,no,yes,unknown,2,sep,1,-1,0,unknown,no


In [55]:
# Q1. 마케팅 응답 고객들의 나이를 10살 단위로 변환 했을 때, 가장 많은 인원을 가진 나이대는? (0~9 : 0 , 10~19 : 10)
df['age_10'] = df['age'].apply(lambda x: x//10)
df['age_10'].value_counts().sort_values(ascending=False)
#print('정답 = ', '30대')

age_10
3    5056
4    3198
5    2244
2    1638
6     460
7     193
8      57
1      18
9       6
Name: count, dtype: int64

In [56]:
# Q2. 마케팅 응답 고객들의 나이를 10살 단위로 변환 했을 때, 가장 많은 나이대 구간의 인원은 몇명인가?
print(5056,'명')

5056 명


In [57]:
# Q3. 나이가 25살 이상 29살 미만인 응답 고객들중 housing컬럼의 값이 yes인 고객의 수는?
df_t = df[(df['age'] >= 25) & (df['age'] <29)] 
df_t = df_t[df_t['housing'] == 'yes']
df_t.shape[0]

504

In [58]:
# Q4. numeric한 값을 가지지 않은 컬럼들중 unique한 값을 가장 많이 가지는 컬럼은?
lst= [] 
for col in df.select_dtypes(exclude='number'):
    target = df[col]
    lst.append([col,target.nunique()])

df_no_num  = pd.DataFrame(lst).sort_values(1,ascending=False)
df_no_num
#정답 = job, month

,0,1
0,job,12
7,month,12
2,education,4
8,poutcome,4
1,marital,3
6,contact,3
3,default,2
4,housing,2
5,loan,2
9,y,2


In [59]:
# Q5. balance 컬럼값들의 평균값 이상을 가지는 데이터를 ID값을 기준으로 내림차순 정렬했을때 상위 100개 데이터의 balance값의 평균은?
df_t = df[df['balance'] >= df['balance'].mean()].sort_values('ID', ascending=False)[:100]
df_t['balance'].mean()

3473.73

In [60]:
# Q6. 가장 많은 광고를 집행했던 날짜는 언제인가? (데이터 그대로 일(숫자),달(영문)으로 표기)
df[['day','month']].value_counts()
#정답 : may 15 

day  month
15   may      301
14   may      283
13   may      257
7    may      239
21   nov      221
             ... 
4    oct        1
30   dec        1
     aug        1
19   mar        1
21   jun        1
Name: count, Length: 303, dtype: int64

In [61]:
# Q7. 데이터의 job이 unknown 상태인 고객들의 age 컬럼 값의 정규성을 검정하고자 한다. 샤피로 검정의 p-value값을 구하여라
## 샤피로-윌크 검정 : 데이터가 정규분포를 따르는지 검정, p-값이 0.05보다 작다면 정규성을 만족하지 않는다고 판단
df_job_unknown = df[df['job'] == 'unknown']

from scipy.stats import shapiro 
shapiro(df_job_unknown['age'])
#정답 : pvalue=0.1961223594771469 

ShapiroResult(statistic=0.978472096015593, pvalue=0.1961223594771469)

In [62]:
# Q8. age와 balance의 상관계수를 구하여라
df[['age', 'balance']].corr()
#Ans : 0.101987

,age,balance
age,1.000000,0.101987
balance,0.101987,1.000000


In [63]:
# Q9. y 변수와 education 변수는 독립인지 카이제곱검정을 통해 확인하려한다. p-value값을 출력하라
from scipy.stats import chi2_contingency
v = pd.crosstab(df['y'], df['education'])
display(v)

chi2 , p , dof, expected = chi2_contingency(v)
print(chi2 , p)

# Ans : 7.901201277473551e-29

education,primary,secondary,tertiary,unknown
y,,,,
no,1424,4555,2559,365
yes,456,1813,1516,182


133.87601246919445 7.901201277473551e-29


In [ ]:
# Q10. 각 job에 따라 divorced/married 인원의 비율을 확인 했을 때 그 값이 가장 높은 값은?
temp = df.pivot_table(index='job', columns='marital', aggfunc='size')
temp['ratio'] = temp['divorced'] / temp['married']
temp.sort_values('ratio', ascending=False)
# Ans : 0.283105

marital,divorced,married,single,ratio
job,,,,
unemployed,62.0,219.0,133.0,0.283105
retired,157.0,572.0,41.0,0.274476
admin.,207.0,762.0,495.0,0.271654
services,138.0,584.0,321.0,0.236301
technician,246.0,1115.0,780.0,0.220628
management,323.0,1580.0,955.0,0.204430
housemaid,49.0,245.0,40.0,0.200000
entrepreneur,44.0,272.0,67.0,0.161765
self-employed,39.0,284.0,131.0,0.137324


### 작업 2유형
데이터 출처 : https://archive.ics.uci.edu/ml/datasets/Bank+Marketing (후처리 작업)  
데이터 설명 : 은행의 전화 마케팅에 대해 고객의 반응 여부  
train : https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/train.csv  
test : https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/test.csv  
submission : https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/submission.csv  

In [75]:
### 모델링 및 submission파일 생성까지 진행할 것 
train= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/train.csv')
test= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/test.csv')
submission= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/submission.csv')

display(train.head())
display(test.head())
display(submission.head())

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome,y
0,13829,29,technician,single,tertiary,no,18254,no,no,cellular,11,may,2,-1,0,unknown,no
1,22677,26,services,single,secondary,no,512,yes,yes,unknown,5,jun,3,-1,0,unknown,no
2,10541,30,management,single,secondary,no,135,no,no,cellular,14,aug,2,-1,0,unknown,no
3,13689,41,technician,married,unknown,no,30,yes,no,cellular,10,jul,1,-1,0,unknown,no
4,11304,27,admin.,single,secondary,no,321,no,yes,unknown,2,sep,1,-1,0,unknown,no


,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome
0,53608,32,management,single,tertiary,no,12569,no,no,cellular,1,jul,2,295,2,success
1,51055,25,services,single,secondary,no,801,no,no,cellular,5,jun,2,-1,0,unknown
2,52573,46,blue-collar,married,secondary,no,1728,yes,no,unknown,26,may,2,-1,0,unknown
3,50458,39,management,divorced,secondary,no,51,no,no,unknown,17,jun,2,-1,0,unknown
4,52272,31,services,single,tertiary,no,1626,no,no,unknown,31,jul,1,-1,0,unknown


,ID,predict
0,53608,0.0
1,51055,0.0
2,52573,0.0
3,50458,0.0
4,52272,0.0


In [80]:
# X, y값 정의 : ID는 무의미한 일련번호이므로 삭제한다. 
y = train['y']
X = train.drop(['ID', 'y'], axis=1)

print(X.shape, y.shape) 

# 카테고리 변수가 있으므로 원-핫 인코딩 한다. 
Xd = pd.get_dummies(X)
Xd

(12870, 15) (12870,)


,age,balance,day,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,29,18254,11,2,-1,0,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
1,26,512,5,3,-1,0,False,False,False,False,...,True,False,False,False,False,False,False,False,False,True
2,30,135,14,2,-1,0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
3,41,30,10,1,-1,0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,27,321,2,1,-1,0,True,False,False,False,...,False,False,False,False,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12865,47,1167,30,1,87,5,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
12866,31,111,21,2,93,2,False,False,False,False,...,False,False,False,True,False,False,True,False,False,False
12867,37,1316,18,1,172,2,False,False,False,False,...,False,False,False,True,False,False,True,False,False,False
12868,42,479,28,2,-1,0,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True


In [81]:
# 훈련/검증 데이터 분리 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(Xd, y, test_size=0.2, stratify= y, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((10296, 50), (2574, 50), (10296,), (2574,))

In [82]:
# 분류문제(반응 y/n)이므로, RandomForestCalssifier 사용 
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

# 훈련 
rf.fit(X_train, y_train)

RandomForestClassifier()

In [ ]:
# 평가 
from sklearn.metrics import roc_auc_score, classification_report

pred = rf.predict_proba(X_test)  ## 0, 1 (반응 N/Y)의 확률값 예측 
display(pred)

print('test roc score : ', roc_auc_score(y_test, pred[:,1]))

array([[0.96, 0.04],
       [0.96, 0.04],
       [0.43, 0.57],
       ...,
       [0.9 , 0.1 ],
       [0.73, 0.27],
       [0.77, 0.23]])

test roc score :  0.7808632949877968


In [88]:
# submission file 
test_pred = rf.predict_proba(pd.get_dummies(test.drop(columns=['ID'])))
display(test_pred)
submission['predict'] = test_pred[:,1]
submission.to_csv('submission.csv',index=False)

array([[0.16, 0.84],
       [0.09, 0.91],
       [0.9 , 0.1 ],
       ...,
       [0.79, 0.21],
       [0.44, 0.56],
       [0.95, 0.05]])